In [1]:
from pyscope.observatory import Observatory
from astropy import coordinates as coord
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)

# Set up after cell logging at debug level
logger = logging.getLogger('pyscope')
logger.setLevel(logging.DEBUG)

In [2]:
rlmt = Observatory(config_path="./rlmt.cfg")
rlmt.connect_all()

DEBUG:pyscope.observatory.observatory:Observatory.__init__() called
DEBUG:pyscope.observatory.observatory:config_path: ./rlmt.cfg
DEBUG:pyscope.observatory.observatory:kwargs: {}
INFO:pyscope.observatory.observatory:Using config file to initialize observatory: ./rlmt.cfg
DEBUG:pyscope.observatory.observatory:Observatory.camera_driver property called
INFO:pyscope.observatory.observatory:Using MaxIm DL as the camera driver
DEBUG:pyscope.observatory.observatory:observatory._import_driver() called
DEBUG:pyscope.observatory.maxim:Maxim.Maxim __init__ called
DEBUG:pyscope.observatory.maxim:_MaximCamera_MaximCamera __init__ called
DEBUG:pyscope.observatory.maxim:Connected setter called with value=True
DEBUG:pyscope.observatory.maxim:_MaximCameraHasFilterWheel called
DEBUG:pyscope.observatory.maxim:Maxim.camera called
DEBUG:pyscope.observatory.maxim:_MaximFilterWheelMaximFilterWheel __init__ called
DEBUG:pyscope.observatory.maxim:Maxim.app called
DEBUG:pyscope.observatory.maxim:_MaximAutofocus

Turning cooler on


True

In [52]:
src = coord.SkyCoord.from_name("HD158659")
print(f"RA: {src.ra.hms},\nDec: {src.dec.dms}")
print(src)
# Print alt az of source
src_altaz = rlmt.get_object_altaz(src)
print(f"Elevation: {src_altaz.alt.deg:.2f}, Azimuth: {src_altaz.az.deg:.2f}")
print(f"Elevation dms: {src_altaz.alt.dms}, \nAzimuth dms: {src_altaz.az.dms}")
if src_altaz.alt.deg < 30:
    raise Exception("Source is too low to observe.")
 
tete_coords = src.transform_to(coord.TETE(obstime = rlmt.observatory_time, location=rlmt.observatory_location))
print(tete_coords)
print(f"RA: {tete_coords.ra.hms},\nDec: {tete_coords.dec.dms}")

RA: hms_tuple(h=17.0, m=30.0, s=59.351923156810926),
Dec: dms_tuple(d=-11.0, m=-11.0, s=-11.719703220001918)
<SkyCoord (ICRS): (ra, dec) in deg
    (262.74729968, -11.18658881)>
Elevation: 47.11, Azimuth: 181.69
Elevation dms: dms_tuple(d=47.0, m=6.0, s=46.09655204027604), 
Azimuth dms: dms_tuple(d=181.0, m=41.0, s=19.886092602544068)
<SkyCoord (TETE: obstime=2024-04-16 11:19:03.723086, location=(-1912384.8991314, -5087338.27905145, 3329720.4999765) m): (ra, dec, distance) in (deg, deg, )
    (263.08680564, -11.20684576, 1.)>
RA: hms_tuple(h=17.0, m=32.0, s=20.83335362252143),
Dec: dms_tuple(d=-11.0, m=-12.0, s=-24.644727417131946)


In [53]:
if src_altaz.alt.deg < 30:
    raise Exception("Source is too low to observe.")
rlmt.recenter(src, 
    target_x_pixel=1024, # TODO: make default center of sensor in each axis
    target_y_pixel=1024,
    exposure=3, 
    save_images=True,
    save_path="./recenter_images/",
    readout=2,
)

INFO:pyscope.observatory.observatory:Attempt 1 of 5
INFO:pyscope.observatory.observatory:Slewing to RA hms_tuple(h=17.0, m=30.0, s=59.351923156810926) and Dec dms_tuple(d=-11.0, m=-11.0, s=-11.719703220001918)
INFO:pyscope.observatory.observatory:Turning on sidereal tracking...
INFO:pyscope.observatory.observatory:Sidereal tracking is on.
INFO:pyscope.observatory.observatory:Attempting to slew to coordinates...
INFO:pyscope.observatory.observatory:Slewing to RA 17.53912 and Dec -11.20685
INFO:pyscope.observatory.observatory:Settling for 3.00 seconds...
INFO:pyscope.observatory.observatory:Settling for 3.00 seconds
INFO:pyscope.observatory.observatory:Taking 3.00 second exposure
INFO:pyscope.observatory.observatory:Exposure complete
INFO:pyscope.observatory.observatory:Using Maxim to save image
INFO:pyscope.observatory.observatory:Overwrite allowed for header keys ['AIRMASS', 'OBJECT', 'TELESCOP', 'INSTRUME', 'OBSERVER']
INFO:pyscope.observatory.observatory:Getting header from MaxIm ima

com_error: (-2147352567, 'Exception occurred.', (65535, 'MaxIm DL 6', 'No matching stars found', None, 0, 0), None)

# Recenter troubleshooting

In [ ]:
# Load image recenter_images\M51-0001_g.fts
import astropy.io.fits as fits
import astropy.wcs
import matplotlib.pyplot as plt
import numpy as np

src = coord.SkyCoord.from_name("M51")
print("Source: M51\n=============================")
print(f"RA: {src.ra.hms},\nDec: {src.dec.dms}")
print(src)
# Print alt az of source
src_altaz = rlmt.get_object_altaz(src)
print(f"Elevation: {src_altaz.alt.deg:.2f}, Azimuth: {src_altaz.az.deg:.2f}")
print(f"Elevation dms: {src_altaz.alt.dms}, \nAzimuth dms: {src_altaz.az.dms}")

hdr = fits.getheader("./recenter_images/M51-0001_g.fts")

wcs = astropy.wcs.WCS(hdr)
print("\nWCS \n=============================")
# Print center value in hms, dms
center = wcs.all_pix2world(1024, 1024, 1)
print(f"Center: {center}")
center = coord.SkyCoord(center[0], center[1], unit="deg")
print(f"Center: {center}")
print(f"Center: {center.ra.hms}, \n{center.dec.dms}")

center_coord = wcs.pixel_to_world(1024, 1024)
print(f"Center: {center_coord}")
print(f"Center: {center_coord.ra.hms}, \n{center_coord.dec.dms}")

print("Slew coordinates")
slew_coords = rlmt.get_object_slew(src)
print(slew_coords)
print(f"RA: {slew_coords.ra.hms},\nDec: {slew_coords.dec.dms}")

print("TETE coordinates")
tete_coords = src.transform_to(coord.TETE(obstime = rlmt.observatory_time, location=rlmt.observatory_location))
print(tete_coords)

print("icrs coordinates")
icrs_coords = src.transform_to(coord.ICRS)
print(icrs_coords)

In [ ]:
rlmt.observatory_time